Helper function The helper function contains all the functions needed

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle



def CameraCalibration(images):
    """Calibrate the camera by finding the image and object points in the chessboard image set"""

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6 * 9, 3), np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
    #print(objp)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.


    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #write_name = 'corners_found'+str(idx)+'.jpg'
            #cv2.imwrite(write_name, img)
            #cv2.imshow('img', img)
            #cv2.waitKey(500)

    #print(imgpoints)
    cv2.destroyAllWindows()

    return objpoints, imgpoints


def ColorandGradient(img, s_thresh=(170, 255), sx_thresh=(70, 255)):

    """Use color transforms, gradients, etc., to create a thresholded binary image."""

    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:, :, 1]
    s_channel = hls[:, :, 2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0)  # Take the derivative in x
    abs_sobelx = np.absolute(sobelx)  # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    threshold_binary = np.zeros_like(sxbinary)
    threshold_binary[((sxbinary == 1) | (s_binary == 1))] = 1
    threshold_binary = np.uint8(255*threshold_binary)
    #color_binary = np.dstack((np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return threshold_binary



def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    # defining a blank mask to start with
    mask = np.zeros_like(img)


    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def bird_eye(img):
    """Apply the perspective transform to the image """

    src = np.float32([[243, 690], [572, 466], [711, 466], [1067, 690]])
    dest = np.float32([[243, 690], [243, 0], [1067, 0], [1067, 690]])

    # Use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dest)

    # Use cv2.warpPerspective() to warp your image to a top-down view
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M

def unwarp(img):
    """Warp the detected lane boundaries back onto the original image """

    src = np.float32([[243, 690], [572, 466], [711, 466], [1067, 690]])
    dest = np.float32([[243, 690], [243, 0], [1067, 0], [1067, 690]])

    # Use cv2.getPerspectiveTransform() to get Minv, the inverse transform matrix
    Minv = cv2.getPerspectiveTransform(dest, src)

    # Use cv2.warpPerspective() to warp your image to a top-down view
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, Minv, img_size, flags=cv2.INTER_LINEAR)
    return warped, Minv




def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0] // 2:, :], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0] // 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0] // nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low),
                      (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low),
                      (win_xright_high, win_y_high), (0, 255, 0), 2)

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    try:
        left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1 * ploty ** 2 + 1 * ploty
        right_fitx = 1 * ploty ** 2 + 1 * ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #**plt.plot(left_fitx, ploty, color='yellow')
    #**plt.plot(right_fitx, ploty, color='yellow')
    return out_img, left_fit, right_fit, left_fitx, right_fitx

def fit_real_world_polynomial(binary_warped, ympp, xmpp ):
    """Calculate the polynomial fit parameters in the real world"""

    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ym_per_pix = ympp # meters per pixel in y dimension
    xm_per_pix = xmpp  # meters per pixel in x dimension

    # Generate x and y values in real world
    ynum = binary_warped.shape[0]
    ploty = np.linspace(0, ynum - 1, num=ynum)
    ploty = ym_per_pix * ploty
    leftx = xm_per_pix*leftx
    lefty = ym_per_pix*lefty
    rightx = xm_per_pix*rightx
    righty = ym_per_pix*righty

    #fit the detected lane lines points in the polynomial equation
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)


    try:
        left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1 * ploty ** 2 + 1 * ploty
        right_fitx = 1 * ploty ** 2 + 1 * ploty
    #Get the lane lines base in the image in the real world version
    left_base = left_fitx[len(ploty)-1]
    right_base = right_fitx[len(ploty)-1]
    return left_fit, right_fit, left_base, right_base



def measure_curvature_pixels(ploty, left_fit, right_fit ):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2 * left_fit[0] * y_eval + left_fit[1]) ** 2) ** 1.5) / np.absolute(2 * left_fit[0])
    right_curverad = ((1 + (2 * right_fit[0] * y_eval + right_fit[1]) ** 2) ** 1.5) / np.absolute(2 * right_fit[0])

    return left_curverad, right_curverad






**Main Detection**
function This function contains the lane detection main process.

In [3]:

def LaneDetection(image):
    """
    Detect the lane lines for the given image.
    This function is the combination of the whole detection process.
    Returns the fit lane lines x positions.
    """
    # TO DO: Apply a distortion to a raw image
    # Read in the saved objpoints and imgpoints
    dist_pickle = pickle.load(open("wide_dist_pickle.p", "rb"))
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    dst = cv2.undistort(image, mtx, dist, None, mtx)

    # TO DO: Use color transforms, gradients, etc., to create a thresholded binary image.
    img = np.copy(dst)
    # Use color transforms and gradients in HLS color space, to create a thresholded binary image
    threshold_binary = ColorandGradient(img, s_thresh=(170, 255), sx_thresh=(30, 255))


    # TO DO: Apply a mask of region of interest
    vertices = np.array([[(100,720), (565,460), (725, 460), (1180, 720)]], dtype=np.int32)
    masked_img = region_of_interest(threshold_binary, vertices)


    # TO DO: Apply a perspective transform to rectify binary image ("birds-eye view")
    warped, M = bird_eye(masked_img)


    # TO DO: Detect lane pixels and fit to find the lane boundary.
    binary_warped = np.copy(warped)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit the polynomial equation and get the parameters for left and right lane lines
    lane_lines, left_fit, right_fit, left_fitx, right_fitx = fit_polynomial(binary_warped)


    # TO DO: Determine the curvature of the lane and vehicle position with respect to center.
    ynum = lane_lines.shape[0]
    ploty = np.linspace(0, ynum-1, num=ynum)
    # Calculate the lane lines curvature at the bottom of the image in real world
    # Convert the pixels into the distance and fit in the polynomial equations
    ympp = 3 / 100  # meters per pixel in y dimension
    xmpp = 3.7 / 800  # meters per pixel in x dimension
    ploty = ympp*ploty # distance in y in real world
    left_fit, right_fit, left_base, right_base = fit_real_world_polynomial(binary_warped, ympp, xmpp)
    # Calculate the curvature in the real world
    left_curverad, right_curverad = measure_curvature_pixels(ploty, left_fit, right_fit)
    avg_curvature = (left_curverad + right_curverad)/2
    #print('The left curvature is ' + str(left_curverad) + ' m, and the right curvature is ' + str(right_curverad) + ' m.')
    # Calculate the lane lines center and the car center
    lane_lines_center = (left_base + right_base)/2
    car_center = xmpp*img_size[0]/2
    # vehicle position with respect to center
    offset = car_center - lane_lines_center
    #print('The vehicle position with respect to the lane lines center is: ' + str(offset) +' m.')


    # TO DO: Warp the detected lane boundaries back onto the original image.
    warped_img = np.zeros_like(img)
    warped_img[lefty, leftx] = [255, 0, 0]
    warped_img[righty, rightx] = [0, 0, 255]
    y = np.linspace(0, ynum-1, num=ynum)

    # Define the lane lines points
    left = np.round(left_fitx)
    right = np.round(right_fitx)
    left_boundary = np.dstack((left, y))
    right_boundary = np.dstack((right, y))
    # Flip the right lane detected points to connect with the left points
    right_boundary = np.flip(right_boundary, axis = 1)
    vertices = np.hstack((left_boundary, right_boundary))
    mask = np.copy(warped_img).astype(np.uint8)
    # Fill the lane part with the green color
    fill_lane = cv2.fillPoly(mask, np.int32(vertices), (0, 200, 0))
    # Add the information text on the image
    cv2.putText(fill_lane,'|',(int(fill_lane.shape[1]/2), fill_lane.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),8)
    cv2.putText(fill_lane,'|',(int(lane_lines_center/xmpp), fill_lane.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),8)
    #cv2.imwrite('output_images/fill_lane_straight_lines1.jpg', fill_lane, None)
    unwarped_lane, Minv = unwarp(fill_lane)
    result = cv2.addWeighted(img, 1., unwarped_lane, 0.6, 0.)


    # TO DO: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
    cv2.putText(result,'Vehicle is ' + str(round(offset,3))+'m'+' with respect to the lane center', (50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),thickness=2)
    cv2.putText(result,'Radius of curvature: '+str(round(avg_curvature))+'m',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),thickness=2)
    #cv2.imwrite('output_images/final_unwarped_straight_lines1.jpg', result, None)
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

    plt.title('Final Result')
    plt.imshow(result)
    plt.axis('off')
    plt.show()
    return result




**Try on Video**

In [5]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import matplotlib.image as mpimg



# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


# TO DO: Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
# Detect the object and image points for camera calibration using the chessboard images
objpoints, imgpoints = CameraCalibration(images)

# Test undistortion on an image
filepath = 'test_images/test1.jpg'
img = cv2.imread(filepath)
img_size = (img.shape[1], img.shape[0])

# Calibrate the camera using the object and image points, and get the camera matrix and distortion parameters
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
print("The camera calibration matrix is: " + str(mtx))
print("The distortion coefficients are :" + str(dist))

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
dist_pickle["objpoints"] = objpoints
dist_pickle["imgpoints"] = imgpoints
pickle.dump( dist_pickle, open( "wide_dist_pickle.p", "wb" ) )


white_output = 'output_images/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_images/project_video.mp4").subclip(0,5)
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(LaneDetection) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

ModuleNotFoundError: No module named 'requests'